In [13]:

# 📌 Dynamic Pricing for Urban Parking Lots
# Summer Analytics 2025 - Capstone Project

# ✅ Install libraries (only needed if not installed)
!pip install pathway bokeh

import numpy as np
import pandas as pd
from datetime import datetime
import time

from bokeh.plotting import figure, output_notebook, show
from bokeh.io import push_notebook
from bokeh.models import ColumnDataSource

import pathway as pw

output_notebook()


In [14]:

# ✅ Load your dataset
DATA_PATH = '/content/dataset.csv'  # Uploaded to Colab root by default

df = pd.read_csv(DATA_PATH)
print("Sample rows:")
df.head()


Sample rows:


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00



## 📌 Project Overview

This notebook implements **Dynamic Pricing** for 14 urban parking lots:
- ✅ Model 1: Baseline Linear Pricing
- ✅ Model 2: Demand-Based Pricing
- ✅ Real-time streaming simulation using Pathway
- ✅ Visualization using Bokeh


In [15]:

# ✅ Vehicle Type Weights
vehicle_type_weight = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5
}


In [16]:

# ✅ Baseline Linear Pricing Model
def baseline_linear_price(prev_price, occupancy, capacity, alpha=2):
    occ_ratio = occupancy / capacity
    new_price = prev_price + alpha * occ_ratio
    return max(5, min(new_price, 20))


In [17]:

# ✅ Demand-Based Pricing Model
def demand_based_price(base_price, occupancy, capacity, queue_length, traffic, special_day, vehicle_type,
                       alpha=0.5, beta=0.2, gamma=0.1, delta=0.3, epsilon=0.4, lambda_=0.5):
    vehicle_weight = vehicle_type_weight.get(vehicle_type.lower(), 1.0)
    demand = (
        alpha * (occupancy / capacity) +
        beta * queue_length +
        gamma * traffic +
        delta * special_day +
        epsilon * vehicle_weight
    )
    demand_norm = demand / 5.0
    price = base_price * (1 + lambda_ * demand_norm)
    return max(5, min(price, 20))


In [21]:
source = ColumnDataSource(data=dict(time=[], price=[]))

p = figure(title="Real-Time Parking Price",
           x_axis_label='Time', y_axis_label='Price ($)',
           x_axis_type='datetime', plot_width=800, plot_height=400)

p.line(x='time', y='price', source=source, line_width=2, color='navy')
handle = show(p, notebook_handle=True)

In [30]:

# ✅ Simulate Real-Time Streaming
base_price = 10
current_price = base_price

# Run for first 50 rows for demo
for idx, row in df.head(50).iterrows():
    occupancy = row['Occupancy']
    capacity = row['Capacity']
    queue_length = row['QueueLength']
    traffic_str = row['TrafficConditionNearby']

    # Convert traffic string to numeric:
    traffic_map = {'low': 0.3, 'medium': 0.6, 'high': 1.0}
    traffic = traffic_map.get(traffic_str.strip().lower(), 0.5)

    special_day = row['IsSpecialDay']
    vehicle_type = row['VehicleType']

    # Toggle model here:
    current_price = baseline_linear_price(current_price, occupancy, capacity)
    # current_price = demand_based_price(base_price, occupancy, capacity, queue_length, traffic, special_day, vehicle_type)

    source.stream(dict(
        time=[datetime.now()],
        price=[current_price]
    ))
    push_notebook(handle=handle)
    time.sleep(0.1)

print("Simulation complete.")



Simulation complete.
